In [1]:
pip install torch torchvision torchaudio --upgrade

     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:39tcmalloc: large alloc 1147494400 bytes == 0x559cd7100000 @  0x7ff56e7c0615 0x559c70f954cc 0x559c7107547a 0x559c70f982ed 0x559c71089e1d 0x559c7100be99 0x559c710069ee 0x559c70f99bda 0x559c7100bd00 0x559c710069ee 0x559c70f99bda 0x559c71008737 0x559c7108ac66 0x559c71007daf 0x559c7108ac66 0x559c71007daf 0x559c7108ac66 0x559c71007daf 0x559c70f9a039 0x559c70fdd409 0x559c70f98c52 0x559c7100bc25 0x559c710069ee 0x559c70f99bda 0x559c71008737 0x559c710069ee 0x559c70f99bda 0x559c71007915 0x559c70f99afa 0x559c71007c0d 0x559c710069ee
     |████████████████████████████████| 881.9 MB 15 kB/s 
     |████████████████████████████████| 23.3 MB 1.4 MB/s 
     |████████████████████████████████| 2.9 MB 74.0 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installa

In [2]:
pip install fastai --upgrade

     |████████████████████████████████| 189 kB 7.4 MB/s 
     |████████████████████████████████| 56 kB 5.9 MB/s 
  Attempting uninstall: fastai
    Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [3]:
!pip install -Uqq fastbook

import fastbook

fastbook.setup_book()

     |████████████████████████████████| 720 kB 8.1 MB/s 
     |████████████████████████████████| 46 kB 5.2 MB/s 
     |████████████████████████████████| 1.2 MB 50.4 MB/s 
     |████████████████████████████████| 51 kB 365 kB/s 
Mounted at /content/gdrive


In [4]:
import os
import zipfile

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from fastai import *
from fastai.text import * 
from fastai.text.all import *
import torch    

import re

from google.colab import drive

In [5]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
os.chdir('gdrive/MyDrive/capstone')

In [7]:
## Reading zip file from my Google Drive

cwd = os.getcwd() # Get current directory

with zipfile.ZipFile("GoT_Script.csv.zip", "r") as zip_ref:
    zip_ref.extractall(cwd)

In [8]:
data = 'GoT_Script.csv'
GoT = pd.read_csv(data)
df = pd.DataFrame(GoT)
pd.set_option('display.max_colwidth', None)
df.head(3)

,release_date,season,episode,episode_title,character,dialogue
0,2011-04-17,1,1,Winter is Coming,Waymar Royce,"What do you expect? They're savages. One lot steals a goat from another lot and before you know it, they're ripping each other to pieces."
1,2011-04-17,1,1,Winter is Coming,Will,"I've never seen wildlings do a thing like this. I've never seen a thing like this, not ever in my life."
2,2011-04-17,1,1,Winter is Coming,Waymar Royce,How close did you get?


In [9]:
df['dialogue'] = df['dialogue'].str.lower()

In [10]:
## Dropping extra characters for supposed improved accuracy

df = df[df.character != 'Extra']
print(len(df))

22665


In [11]:
## Dropping group characters for supposed improved accuracy

df = df[df.character != 'Group']
print(len(df))

22554


In [12]:
df = df[df.character != 'Morgans Friend']
print(len(df))

22552


In [13]:
## Dropping characters with low value counts

counts = df['character'].value_counts()
df = df.loc[df['character'].isin(counts.index[counts > 14])]
len(df)

21827

In [14]:
df = df[['dialogue', 'character']]

In [15]:
df.head(1)

,dialogue,character
15,go on. father's watching.,Jon Snow


In [16]:
len(df)

21827

In [17]:
data_lm = TextDataLoaders.from_df(df, text_col='dialogue', label_col='character', valid_pct=0.2, is_lm=True, bs=64)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [18]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3, metrics=[accuracy, Perplexity()]).to_fp16()
learn.fit_one_cycle(1, 1e-2)

/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.323205,3.924455,0.281114,50.625462,09:03


In [19]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3)

/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.825604,3.783871,0.296997,43.985970,13:00
1,3.711927,3.682844,0.306193,39.759296,12:58
2,3.578026,3.619782,0.313192,37.329437,12:52
3,3.400601,3.614211,0.313314,37.122055,12:48
4,3.221246,3.632208,0.312636,37.796192,12:51
5,3.019427,3.682082,0.311537,39.729042,12:57
6,2.847013,3.734550,0.304430,41.869160,12:55
7,2.688047,3.795933,0.304009,44.519775,12:51
8,2.583900,3.826860,0.303263,45.918114,12:50
9,2.533255,3.835797,0.302585,46.330353,12:49


In [20]:
import warnings
warnings.filterwarnings("ignore")

In [21]:
learn.predict("xxbos", n_words=50, temperature=0.75)

"please . your grace . i 'm not a fighter . that 's the fighter . this is death , no , no , no . your death was a terrible death . when will the starks make amends for our past ? the gods do"

In [22]:
learn.save_encoder('final_finetuned_encoder')

In [23]:
dls_clas = TextDataLoaders.from_df(df, 
                        valid_pct = 0.3, 
                        text_col = 'dialogue',
                        label_col = 'character', 
                        bs = 64, 
                        text_vocab = data_lm.vocab)

In [24]:
learn_clf = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult = 0.5, metrics = accuracy).to_fp16()
learn_clf = learn_clf.load_encoder('final_finetuned_encoder')

In [27]:
learn_clf.fit_one_cycle(1, 2e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.652032,4.563701,0.080024,05:18


In [28]:
learn_clf.freeze_to(-2)
learn_clf.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2)) 

epoch,train_loss,valid_loss,accuracy,time
0,4.152142,4.058271,0.098809,05:42


In [29]:
learn_clf.freeze_to(-3)
learn_clf.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.010125,3.961652,0.115302,09:14


In [30]:
learn_clf.unfreeze()
learn_clf.fit_one_cycle(5, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.839547,3.935665,0.118509,13:10
1,3.823244,3.912616,0.123091,13:08
2,3.692720,3.894151,0.125076,13:04
3,3.623873,3.883689,0.124160,13:05
4,3.571728,3.887533,0.124313,13:10


In [31]:
learn_clf.predict("you are not my queen")

('Jorah Mormont',
 TensorText(47),
 TensorText([9.4393e-05, 1.2216e-04, 9.0413e-06, 4.8012e-03, 5.1707e-03, 6.8191e-03, 3.8363e-05, 1.1785e-03, 1.8939e-05, 3.9977e-04, 1.1113e-03, 1.5312e-02, 3.3336e-04, 6.9741e-05, 1.7291e-05, 4.0248e-03,
         8.6065e-03, 1.0915e-05, 9.9897e-02, 1.2485e-01, 2.5635e-05, 5.1695e-02, 1.1617e-04, 9.0460e-04, 1.4329e-03, 8.7407e-04, 2.0549e-03, 8.5782e-04, 1.7365e-03, 2.0795e-03, 3.7024e-04, 3.0513e-04,
         5.3624e-04, 1.3023e-01, 5.0471e-03, 2.0950e-06, 2.8011e-05, 2.6608e-04, 1.0358e-04, 2.9715e-05, 1.1193e-02, 5.1522e-04, 5.8540e-03, 3.3031e-05, 1.1844e-04, 2.9662e-05, 1.5416e-02, 1.8539e-01,
         7.4156e-05, 5.7501e-06, 1.0293e-03, 7.7393e-04, 9.6977e-04, 2.6792e-05, 1.5711e-04, 7.8849e-04, 1.5314e-05, 3.2258e-04, 1.1489e-05, 2.8380e-04, 3.2636e-04, 3.7554e-04, 1.3195e-04, 7.4420e-05,
         8.8780e-05, 9.1290e-05, 3.8007e-03, 3.7602e-05, 2.9732e-03, 7.2094e-05, 3.2291e-02, 7.0176e-05, 3.0096e-04, 1.5946e-04, 1.2424e-02, 3.7518e-04, 3.84

In [32]:
learn_clf.save_encoder('final_finetuned_classifier')

In [62]:
user_input_text = input("Enter a few words to generate dialogue: ").lower()

def generate_text(user_input_text):
    
    ## Set number of words and sentences you want in output
    n_words = 50
    n_sentences = 1

    ## Predict on language model
    prediction = [learn.predict(user_input_text, n_words, temperature=0.75) for _ in range(n_sentences)]
    predicted_sentence = " ".join(prediction)

    ## Predict character speaking using classifier model, index 0 for character name
    character_speaking = learn_clf.predict(predicted_sentence)[0]

    ## Print character and dialogue
    print(f'{character_speaking} : {predicted_sentence}')

generate_text(user_input_text)

Enter a few words to generate dialogue: he is not the true king


Jaime Lannister : he is not the true king . i 'm not a knight . i 'm not the only one here who 's going to accept this place . if it were n't for the lannister who had slaughtered you , it would have been the one who murdered our father . no
